## Alex Khvatov Midterm project

The dataset I selected for this project comes from https://archive.ics.uci.edu/dataset/45/heart+disease

This project represents a classification problem given the criteria.

    For this project we will try to use 

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

pd.options.mode.copy_on_write = True

### Dataset background:

This database contains 76 attributes, but all published experiments refer to using a subset of 14 of them.

In particular, the Cleveland database is the only one that has been used by ML researchers to data and this is what we are going to use for this midterm project.


The "goal" field refers to the presence of heart disease in the patient.  It is integer valued from 0 (no presence) to 4. Experiments with the Cleveland database have concentrated on simply attempting to distinguish presence (values 1,2,3,4) from absence (value 0).  


One file has been "processed", that one containing the Cleveland database.  All four unprocessed files also exist in this directory.

Attribute Information:
* Only 14 used
    1. #3  (age)       
    2. #4  (sex)       
    3. #9  (cp) - chest pain        
    4. #10 (trestbps)  resting blood pressure
    5. #12 (chol)      cholesterol
    6. #16 (fbs)       fasting blood sugar
    7. #19 (restecg)   resting electrocardiographic results 
    8. #32 (thalach)   maximum heart rate achieved
    9. #38 (exang)     exercise induced angina (1 = yes; 0 = no)
    10. #40 (oldpeak)  ST depression induced by exercise relative to rest
    11. #41 (slope)    the slope of the peak exercise ST segment 
    12. #44 (ca)       number of major vessels (0-3) colored by flourosopy 
    13. #51 (thal)     3 = normal; 6 = fixed defect; 7 = reversable defect (A blood disorder called thalassemia)
    14. #58 (num)       (the predicted attribute)

<details>
       <summary>Complete attribute documentation</summary>
       
      3 age: age in years
      4 sex: sex (1 = male; 0 = female)
      9 cp: chest pain type
        -- Value 1: typical angina
        -- Value 2: atypical angina
        -- Value 3: non-anginal pain
        -- Value 4: asymptomatic
     10 trestbps: resting blood pressure (in mm Hg on admission to the 
        hospital)
     12 chol: serum cholestoral in mg/dl
     16 fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
     19 restecg: resting electrocardiographic results
        -- Value 0: normal
        -- Value 1: having ST-T wave abnormality (T wave inversions and/or ST 
                    elevation or depression of > 0.05 mV)
        -- Value 2: showing probable or definite left ventricular hypertrophy
                    by Estes' criteria
     32 thalach: maximum heart rate achieved
     38 exang: exercise induced angina (1 = yes; 0 = no)
     40 oldpeak = ST depression induced by exercise relative to rest
     41 slope: the slope of the peak exercise ST segment
        -- Value 1: upsloping
        -- Value 2: flat
        -- Value 3: downsloping
     44 ca: number of major vessels (0-3) colored by flourosopy
     51 thal: 3 = normal; 6 = fixed defect; 7 = reversable defect (A blood disorder called thalassemia)
     58 num: diagnosis of heart disease (angiographic disease status)
        -- Value 0: < 50% diameter narrowing
        -- Value 1: > 50% diameter narrowing
        (in any major vessel: attributes 59 through 68 are vessels)
     59 lmt
     60 ladprox
     61 laddist
     62 diag
     63 cxmain
     64 ramus
     65 om1
     66 om2
     67 rcaprox
     68 rcadist
     69 lvx1: not used
     70 lvx2: not used
     71 lvx3: not used
     72 lvx4: not used
     73 lvf: not used
     74 cathef: not used
     75 junk: not used
     76 name: last name of patient 
	(I replaced this with the dummy string "name")
    
    Missing Attribute Values: Several.  Distinguished with value -9.0.
</details>

## Data Preparation

In [50]:
#To make it easier to look up the meanings of columns we are going to create a dictionary of columns-explanations.
column_meanings = {'age':'Age', 
                   'sex': 'Gender',
                   'cp': 'Chest Pain',
                   'trestbps': 'Resting Blood Pressure',
                   'chol' : 'Cholesterol',
                   'fbs':'Fasting blood sugar',
                   'restecg':'Resting electrocardiographic results',
                   'thalach':'Maximum heart rate achieved',
                   'exang':'Exercise induced angina (1 = yes; 0 = no)',
                   'oldpeak': 'ST depression induced by exercise relative to rest',
                   'slope':'The slope of the peak exercise ST segment',
                   'ca':'Number of major vessels (0-3) colored by flourosopy',
                   'thal':'3 = normal; 6 = fixed defect; 7 = reversable defect (A blood disorder called thalassemia)',
                   'num':'The predicted attribute (heart disease diagnosis)'
                  }

In [2]:
#Columns defined - I will assign a more readable column names in place of abbreviated ones

columns = ['age', 'sex', 'cp', 'trestbps', 'chol','fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num']

pwd = Path.resolve(Path("./data/processed.cleveland.data"))

p_cleveland_df = pd.read_csv(pwd, header=None, names= columns )
p_cleveland_df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


Now we need to make sure we deal with missing data

In [3]:
#Check for null values
p_cleveland_df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
num         0
dtype: int64

In [4]:
string_columns = list(p_cleveland_df.dtypes[p_cleveland_df.dtypes == 'object'].index)
for c in string_columns:
    p_cleveland_df[c] = p_cleveland_df[c].str.strip()
    

In [5]:
for c in columns:
    col_vals = list(p_cleveland_df[c].values)
    if('?' in col_vals):
        print(f"Missing data in {c} = {col_vals.count('?')}")


Missing data in ca = 4
Missing data in thal = 2


In [6]:
#Since we cannot impute the value for categorical value column - we are going to drop the missing 'thal' rows
p_cleveland_df = p_cleveland_df[p_cleveland_df['thal']!='?']

#Missing data for 'ca' column (number of major vessels (0-3) colored by flourosopy) we are going to set to 0
p_cleveland_df.loc[p_cleveland_df['ca']=='?', 'ca'] = 0.0
p_cleveland_df



,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


In [7]:
list(p_cleveland_df.columns)

['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal',
 'num']

In [8]:
p_cleveland_df.nunique()

age          41
sex           2
cp            4
trestbps     50
chol        152
fbs           2
restecg       3
thalach      91
exang         2
oldpeak      40
slope         3
ca            5
thal          3
num           5
dtype: int64

In [9]:
p_cleveland_df.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
num         0
dtype: int64

In [10]:
p_cleveland_df.dtypes

age         float64
sex         float64
cp          float64
trestbps    float64
chol        float64
fbs         float64
restecg     float64
thalach     float64
exang       float64
oldpeak     float64
slope       float64
ca           object
thal         object
num           int64
dtype: object

In [11]:
p_cleveland_df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45.0,1.0,1.0,110.0,264.0,0.0,0.0,132.0,0.0,1.2,2.0,0.0,7.0,1
299,68.0,1.0,4.0,144.0,193.0,1.0,0.0,141.0,0.0,3.4,2.0,2.0,7.0,2
300,57.0,1.0,4.0,130.0,131.0,0.0,0.0,115.0,1.0,1.2,2.0,1.0,7.0,3
301,57.0,0.0,2.0,130.0,236.0,0.0,2.0,174.0,0.0,0.0,2.0,1.0,3.0,1


In [12]:
#all columns can be converted to type int
int_columns = ['age', 'sex', 'cp', 'trestbps', 'chol','fbs', 'restecg', 'thalach', 'exang', 'slope', 'ca', 'thal', 'num']
float_columns = ['oldpeak']


for c in int_columns:
    p_cleveland_df[c] = pd.to_numeric(p_cleveland_df[c], errors='coerce')
    p_cleveland_df[c] = p_cleveland_df[c].astype(int)


In [13]:
p_cleveland_df.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
num           int64
dtype: object

In [14]:
p_cleveland_df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0,7,1
299,68,1,4,144,193,1,0,141,0,3.4,2,2,7,2
300,57,1,4,130,131,0,0,115,1,1.2,2,1,7,3
301,57,0,2,130,236,0,2,174,0,0.0,2,1,3,1


In [15]:
#At this point we have 'clean' dataset.
#Because 'num' can be 0 - for healthy and other numbers for having presence of a heart disease we can change all the numbers to 1
p_cleveland_df.loc[p_cleveland_df['num']!=0, 'num'] = 1

In [16]:
p_cleveland_df.nunique()
#Based on the above output we can identify 'age', 'trestbps', 'chol', 'thalach', 'oldpeak' as numeric columns and the rest as categorical


age          41
sex           2
cp            4
trestbps     50
chol        152
fbs           2
restecg       3
thalach      91
exang         2
oldpeak      40
slope         3
ca            4
thal          3
num           2
dtype: int64

In [17]:
p_cleveland_df.head().T

,0,1,2,3,4
age,63.0,67.0,67.0,37.0,41.0
sex,1.0,1.0,1.0,1.0,0.0
cp,1.0,4.0,4.0,3.0,2.0
trestbps,145.0,160.0,120.0,130.0,130.0
chol,233.0,286.0,229.0,250.0,204.0
fbs,1.0,0.0,0.0,0.0,0.0
restecg,2.0,2.0,2.0,0.0,2.0
thalach,150.0,108.0,129.0,187.0,172.0
exang,0.0,1.0,1.0,0.0,0.0
oldpeak,2.3,1.5,2.6,3.5,1.4


In [18]:
from sklearn.model_selection import train_test_split

In [19]:
df_full_train, df_test = train_test_split(p_cleveland_df, test_size = 0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state=1)

In [20]:
df_train.reset_index(drop=True, inplace=True)
df_val.reset_index(drop=True, inplace=True)
df_test.reset_index(drop=True, inplace=True)

In [21]:
y_train = df_train.num.values
y_val = df_val.num.values
y_test = df_test.num.values

In [22]:
del df_train['num']
del df_val['num']
del df_test['num']

## EDA (Exploratory Data Analysis)

In [23]:
df_full_train = df_full_train.reset_index(drop=True)


In [24]:
df_full_train.num.value_counts(normalize=True)

num
0    0.529167
1    0.470833
Name: proportion, dtype: float64

In [25]:
#in our train dataset 47% of the patients have been diagnosed with heart disease (heart disease rate)

In [26]:
global_heart_disease_rate = df_full_train.num.mean()
round(global_heart_disease_rate, 2)

np.float64(0.47)

In [27]:
# the reason mean gives churn rate is b/c in binary viarable is xi is 0 or 1 the sum of ones gives mean

**Mean** 
$$M=1/n\sum_{i=1}^n x_i$$

In [28]:
df_full_train.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
ca            int64
thal          int64
num           int64
dtype: object

In [39]:
categorical_columns = ['sex', 'cp', 'fbs', 'restecg',  'exang',  'slope', 'ca', 'thal']
numeric_columns = ['age', 'trestbps', 'chol', 'thalach','oldpeak']

## Heart Disease Rate and Risk Factor

In [40]:
# sex (1 = male; 0 = female)
heart_disease_in_males = df_full_train[df_full_train.sex == 1].num.mean()
heart_disease_in_females = df_full_train[df_full_train.sex == 0].num.mean()
print(f"{heart_disease_in_males=} \n{heart_disease_in_females=}")
print(f"{global_heart_disease_rate=}")

heart_disease_in_males=np.float64(0.5679012345679012) 
heart_disease_in_females=np.float64(0.2692307692307692)
global_heart_disease_rate=np.float64(0.4708333333333333)


In [57]:
for c in categorical_columns:
    print(f"{column_meanings[c].center(30)}")
    df_group = df_full_train.groupby(c).num.agg(['mean', 'count'])
    df_group['diff'] = df_group['mean'] - global_heart_disease_rate
    df_group['risk'] = df_group['mean'] / global_heart_disease_rate
    display(df_group)
    print()
    print()

            Gender            


,mean,count,diff,risk
sex,,,,
0,0.269231,78,-0.201603,0.571818
1,0.567901,162,0.097068,1.206162




          Chest Pain          


,mean,count,diff,risk
cp,,,,
1,0.368421,19,-0.102412,0.782487
2,0.189189,37,-0.281644,0.401818
3,0.231884,69,-0.238949,0.492497
4,0.721739,115,0.250906,1.532897




     Fasting blood sugar      


...

,mean,count,diff,risk
slope,,,,
1,0.280702,114,-0.190132,0.596181
2,0.636364,110,0.165530,1.351569
3,0.687500,16,0.216667,1.460177




Number of major vessels (0-3) colored by flourosopy


,mean,count,diff,risk
ca,,,,
0,0.283688,141,-0.187145,0.602523
1,0.685185,54,0.214352,1.455261
2,0.806452,31,0.335618,1.712818
3,0.785714,14,0.314881,1.668774




3 = normal; 6 = fixed defect; 7 = reversable defect (A blood disorder called thalassemia)


,mean,count,diff,risk
thal,,,,
3,0.248120,133,-0.222713,0.526981
6,0.583333,12,0.112500,1.238938
7,0.768421,95,0.297588,1.632045


### Conclusion

1. Males are more likely to be diagnosed with heart disease than females
2. Asymptomatic chest pain leads to heart disease than other types of chest pain
3. Higher fasting blood sugar leads to heart disease, but not sugnificantly
4. Patinet with resting electrocardiographic results (Value 1:) having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV) are at sugnificant risk to be diagnosed
5. Exercise induced angina if identified (1) has a higher risk of being diagnosed with heart disease
6. The slope of the peak exercise ST segment when flat (2) is at moderat risk of being diagnosed
7. When number of major vessels (0-3) colored by flourosopy is higher - the risk is higher than none (0). NOTE: this column had 4 values missing and replaced with 0, which may lead to a skewed results.
8. Higher risk of having a diagnosis for patients with a thalassemia - a presence of a disorder (values other than 3).

## Mutual information as a measure of relative importance of categorical variables

In [45]:
from sklearn.metrics import mutual_info_score

In [47]:
def mutual_info_heart_disease_score(series):
    return mutual_info_score(series, df_full_train.num)

In [49]:
mi = df_full_train[categorical_columns].apply(mutual_info_heart_disease_score)
mi.sort_values(ascending=False)

thal       0.132791
cp         0.125540
ca         0.107134
exang      0.084066
slope      0.067643
sex        0.040504
restecg    0.023130
fbs        0.001044
dtype: float64